In [1]:
import pandas as pd
import numpy as np
np.float_ = np.float64
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from dotenv import load_dotenv
import os
import functools

load_dotenv()

True

In [2]:
ELK_HOST = os.getenv("ELK_HOST")

es = Elasticsearch(hosts=[ELK_HOST], timeout=300, max_retries=10, retry_on_timeout=True, verify_certs=False, ssl_show_warn=False)

In [3]:
index_list_names = [
    "loadtest-webrtc-2024-kurento-2p",
    "loadtest-webrtc-2024-kurento-8p",
    "loadtest-webrtc-2024-kurento-3p-10s",
    "loadtest-webrtc-2024-kurento-3p-40s",
    "loadtest-webrtc-2024-pion-2p",
    "loadtest-webrtc-2024-pion-8p",
    "loadtest-webrtc-2024-pion-3p-10s",
    "loadtest-webrtc-2024-pion-3p-40s",
    "loadtest-webrtc-2024-mediasoup-2p",
    "loadtest-webrtc-2024-mediasoup-8p",
    "loadtest-webrtc-2024-mediasoup-3p-10s",
    "loadtest-webrtc-2024-mediasoup-3p-40s"
]

test_times = [
    {
        "min": "2024-06-18 14:57:24.618000+02:00",
        "max": "2024-06-18 15:05:54.426000+02:00",
    },
    {
        "min": "2024-06-18 15:08:46.655000+02:00",
        "max": "2024-06-18 15:12:21.692000+02:00"
    },
    {
        "min": "2024-06-18 14:31:51.541000+02:00",
        "max": "2024-06-18 14:37:41.897000+02:00"
    },
    {
        "min": "2024-06-18 14:45:59.777000+02:00",
        "max": "2024-06-18 14:51:47.771000+02:00"
    },
    {
        "min": "2024-06-13 15:53:20.001000+02:00",
        "max": "2024-06-13 16:05:24.229000+02:00"
    },
    {
        "min": "2024-08-16 12:13:36.601000+02:00",
        "max": "2024-08-16 12:19:44.806000+02:00"
    },
    {
        "min": "2024-06-13 15:41:35.494000+02:00",
        "max": "2024-06-13 15:49:38.492000+02:00"
    },
    {
        "min": "2024-06-13 14:20:27.960000+02:00",
        "max": "2024-06-13 14:28:20.802000+02:00"
    },
    {
        "min": "2024-08-16 10:14:20.912000+02:00",
        "max": "2024-08-16 10:32:05.237000+02:00"
    },
    {
        "min": "2024-08-14 10:43:54.881000+02:00",
        "max": "2024-08-14 10:47:35.287000+02:00"
    },
    {
        "min": "2024-08-16 10:47:57.575000+02:00",
        "max": "2024-08-16 11:01:32.660000+02:00"
    },
    {
        "min": "2024-08-16 11:09:37.743000+02:00",
        "max": "2024-08-16 11:19:08.166000+02:00"
    },
]

In [4]:
from datetime import datetime
node_types = ["browseremulator", "masternode", "medianode"]
for i, index_name in enumerate(index_list_names):
    current_time = pd.Timestamp.now().isoformat()
    min = test_times[i]["min"]
    max = test_times[i]["max"]
    date_format = "%Y-%m-%d %H:%M:%S.%f%z"
    min = datetime.strptime(min, date_format)
    max = datetime.strptime(max, date_format)
    for node_type in node_types:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {"fields.node_role": node_type}},
                        {
                            "range": {
                                "@timestamp": {
                                    "lte": max.isoformat(),
                                    "gte": min.isoformat(),
                                }
                            }
                        },
                    ]
                }
            }
        }
        rel = scan(
            client=es,
            query=query,
            scroll="8h",
            index="metric*",
            raise_on_error=True,
            preserve_order=False,
            clear_scroll=True,
            request_timeout=300,
        )

        # We need only '_source', which has all the fields required.
        # This elimantes the elasticsearch metdata like _id, _type, _index.
        def generate_data():
            i = 0
            for hit in rel:
                i += 1
                print("Data read: ", i, end="\r")
                data = hit["_source"]
                data_to_save = {"@timestamp": data["@timestamp"]}
                if "system" in data:
                    data = data["system"]
                    if "cpu" in data:
                        data_to_save["cpu"] = data["cpu"]["total"]["norm"]["pct"]
                    if "memory" in data:
                        data_to_save["memory"] = data["memory"]["used"]["pct"]
                if "cpu" in data or "memory" in data:
                    yield data_to_save

        print(f"{current_time} - Processing {index_name}-{node_type}")
        data_generator = generate_data()
        # Create a dataframe.
        df = pd.DataFrame(data_generator)
        if not df.empty:
            df = df.groupby("@timestamp", as_index=False).mean()
            df["@timestamp"] = pd.to_datetime(df["@timestamp"])
            df.to_csv(
                f"dfs_final/{index_name}-{node_type}.csv", index=False
            )
            current_time = pd.Timestamp.now().isoformat()
            print(f"{current_time} - Saved {index_name}-{node_type}.csv")

2024-09-26T11:16:35.589050 - Processing loadtest-webrtc-2024-kurento-2p-browseremulator
2024-09-26T11:17:07.863199 - Saved loadtest-webrtc-2024-kurento-2p-browseremulator.csv
2024-09-26T11:17:07.863199 - Processing loadtest-webrtc-2024-kurento-2p-masternode
2024-09-26T11:17:08.539257 - Saved loadtest-webrtc-2024-kurento-2p-masternode.csv
2024-09-26T11:17:08.539257 - Processing loadtest-webrtc-2024-kurento-2p-medianode
2024-09-26T11:17:09.091098 - Saved loadtest-webrtc-2024-kurento-2p-medianode.csv
2024-09-26T11:17:09.091098 - Processing loadtest-webrtc-2024-kurento-8p-browseremulator
2024-09-26T11:17:17.485370 - Saved loadtest-webrtc-2024-kurento-8p-browseremulator.csv
2024-09-26T11:17:17.485370 - Processing loadtest-webrtc-2024-kurento-8p-masternode
2024-09-26T11:17:17.884742 - Saved loadtest-webrtc-2024-kurento-8p-masternode.csv
2024-09-26T11:17:17.884742 - Processing loadtest-webrtc-2024-kurento-8p-medianode
2024-09-26T11:17:18.285738 - Saved loadtest-webrtc-2024-kurento-8p-medianod